# Car Milage Prediction on MPG dataset using Keras

In [3]:
import pandas as pd
import numpy as np
import keras

from sklearn import pipeline,preprocessing,metrics,model_selection,ensemble, impute
from sklearn_pandas import DataFrameMapper

In [2]:
data=pd.read_csv('mpg_data_example.csv')

## Prepare data

We will use sklearn for processing preprocessing out data

In [46]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [154]:
data.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

In [5]:
data.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

In [20]:
data['origin'].nunique()

3

In [131]:
mapper = DataFrameMapper([
                        (['horsepower'],impute.SimpleImputer()),
        (['cylinders','displacement','weight','acceleration','model year'], preprocessing.MinMaxScaler(feature_range=(0, 1))),
                        (['origin'], preprocessing.OneHotEncoder()),
                        
                        ])

In [132]:
pipeline_obj = pipeline.Pipeline([
    ('mapper',mapper),
    
])

In [133]:
transformed_data = pipeline_obj.fit_transform(data)

In [134]:
pd.DataFrame(transformed_data)

,0,1,2,3,4,5,6,7,8
0,130.0,1.0,0.617571,0.536150,0.238095,0.0,1.0,0.0,0.0
1,165.0,1.0,0.728682,0.589736,0.208333,0.0,1.0,0.0,0.0
2,150.0,1.0,0.645995,0.516870,0.178571,0.0,1.0,0.0,0.0
3,150.0,1.0,0.609819,0.516019,0.238095,0.0,1.0,0.0,0.0
4,140.0,1.0,0.604651,0.520556,0.148810,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
393,86.0,0.2,0.186047,0.333711,0.452381,1.0,1.0,0.0,0.0
394,52.0,0.2,0.074935,0.146583,0.988095,1.0,0.0,1.0,0.0
395,84.0,0.2,0.173127,0.193365,0.214286,1.0,1.0,0.0,0.0
396,79.0,0.2,0.134367,0.286929,0.630952,1.0,1.0,0.0,0.0


In [135]:
# check shape
transformed_data.shape


(398, 9)

In [136]:
# We need only the origin and the scaled values

scaled_features = transformed_data[:,1:]

In [137]:
target_scaler = preprocessing.MinMaxScaler()
target_scaler.fit(data['mpg'].values.reshape(-1,1))

MinMaxScaler()

In [138]:
target = target_scaler.transform(data['mpg'].values.reshape(-1,1))

In [139]:
# split the features
X_train, X_test, y_train, y_test = model_selection.train_test_split(scaled_features, target, test_size=0.20, random_state=42)

## Making the Keras Model

In [220]:

model = keras.Sequential()
model.add(keras.layers.Dense(X_train.shape[1], kernel_initializer='normal', activation="relu"))
model.add(keras.layers.Dense(8, kernel_initializer='normal', activation="relu"))
model.add(keras.layers.Dense(4,kernel_initializer='normal',  activation = 'relu'))
model.add(keras.layers.Dense(1,kernel_initializer='normal',  activation = 'linear'))
model.compile(optimizer = 'adam',loss='mse')

In [221]:
history = (model.fit(X_train, y_train, epochs=100, batch_size=4, validation_data=[X_test, y_test]))

Train on 318 samples, validate on 80 samples
Epoch 1/100
318/318 [==============================] - 1s 3ms/step - loss: 0.1553 - val_loss: 0.0826
Epoch 2/100
318/318 [==============================] - 0s 777us/step - loss: 0.0561 - val_loss: 0.0317
Epoch 3/100
318/318 [==============================] - 0s 868us/step - loss: 0.0293 - val_loss: 0.0165
Epoch 4/100
318/318 [==============================] - 0s 734us/step - loss: 0.0179 - val_loss: 0.0114
Epoch 5/100
318/318 [==============================] - 0s 751us/step - loss: 0.0140 - val_loss: 0.0089
Epoch 6/100
318/318 [==============================] - 0s 728us/step - loss: 0.0123 - val_loss: 0.0080
Epoch 7/100
318/318 [==============================] - 0s 784us/step - loss: 0.0113 - val_loss: 0.0075
Epoch 8/100
318/318 [==============================] - 0s 817us/step - loss: 0.0107 - val_loss: 0.0065
Epoch 9/100
318/318 [==============================] - 0s 762us/step - loss: 0.0097 - val_loss: 0.0062
Epoch 10/100
318/318 [========

### Making predictions on the test set

In [147]:
pred = model.predict(X_test)

Now that we have our model working, we need to prepare some functions.
These functions will handle the data preparations for our model in the web app.
But first we need to save the model, the pipeline that prepares the data and the scaler for the target variable.

We will load all these objects in our web app.
For keras model we will use the inbuilt save functionality. For sklearn object we will use pickle file format using joblib.


#### Let's save all objects.


In [209]:
# save the keras model
model.save('MPG_keras_NN')

# save the pipeline and teh target scaler using joblib and .pkl format
import joblib
joblib.dump(pipeline_obj,'data_pipeline.pkl')
joblib.dump(target_scaler,'target_scaler.pkl')

['target_scaler.pkl']

#### The functions to handle data preparation and making predictions

In [210]:
def get_input_set(input_dict, loaded_pipeline):
    # prepare input data
    temp_data=pd.DataFrame({'x':input_dict}).transpose()
    temp_data = temp_data.apply(pd.to_numeric)
    temp_scaled = loaded_pipeline.transform(temp_data)[:,1:]
    
    return temp_scaled
    
    


In [211]:
model.predict(temp_scaled)

array([[-0.0031298]], dtype=float32)

In [212]:
target_scaler.inverse_transform(model.predict(get_input_set(input_dict, pipeline_obj)))

array([[8.882319]], dtype=float32)

In [213]:
def get_prediction(loaded_model, loaded_target_scaler, input_feature_set):
    return float(np.squeeze(loaded_target_scaler.inverse_transform(loaded_model.predict(input_feature_set))))
    

In [214]:
get_prediction(model, target_scaler, temp_scaled)

8.882319450378418

#### Let us load the saved model and the objects to verify if everything is still working

In [215]:
loaded_model = keras.models.load_model('MPG_keras_NN')

In [217]:
loaded_pipeline = joblib.load('data_pipeline.pkl')
loaded_target_scaler = joblib.load('target_scaler.pkl')

In [219]:
get_input_set(input_dict, loaded_pipeline)

array([[1.        , 0.61757106, 0.7916076 , 0.23809524, 0.        ,
        1.        , 0.        , 0.        ]])

In [218]:
get_prediction(loaded_model, loaded_target_scaler, temp_scaled)


8.882319450378418